In [5]:
import pandas as pd
import json

# JSON 파일 읽기
with open('./visitlog.json', 'r', encoding='utf-8') as file:
    visit_data = json.load(file)

with open('./api.json', 'r', encoding='utf-8') as file:
    api_data = json.load(file)

# visit_data가 단일 객체가 아닌 리스트인 경우를 대비
if isinstance(visit_data, dict):
    visit_data = [visit_data]

# 필요한 데이터를 추출하기 위해 딕셔너리 형태로 변환 (restaurantId를 키로 설정)
api_content = {item['id']: item for item in api_data['content']}

# 데이터프레임 생성 준비
final_data = []

for visit in visit_data:
    rest_id = visit['restaurantId']
    
    # 해당 restaurantId에 대한 apilog 데이터 가져오기
    if rest_id in api_content:
        api_entry = api_content[rest_id]
        
        # 필요한 데이터 추출
        user_rating = api_entry.get('userRating')
        travel_time = api_entry.get('travelTime')
        rating = api_entry.get('rating')
        
        # 메뉴 최대 3개 추출
        menu_items = api_entry.get('menu', [])
        menu1 = menu_items[0] if len(menu_items) > 0 else None
        menu2 = menu_items[1] if len(menu_items) > 1 else None
        menu3 = menu_items[2] if len(menu_items) > 2 else None
        
        # 카테고리 최대 2개 id 추출
        category_ids = [cat['id'] for cat in api_entry.get('categories', [])]
        category_id1 = category_ids[0] if len(category_ids) > 0 else None
        category_id2 = category_ids[1] if len(category_ids) > 1 else None
        
        # visit 정보와 결합
        final_data.append({
            '유저 아이디': visit['userId'],
            '방문한 레스토랑 아이디': rest_id,
            '방문일시': visit['visitedAt'],
            'userRating': user_rating,
            'travelTime': travel_time,
            'rating': rating,
            'menu1': menu1,
            'menu2': menu2,
            'menu3': menu3,
            'category_id1': category_id1,
            'category_id2': category_id2
        })

# 데이터프레임 생성
df = pd.DataFrame(final_data)

# 결과 출력
print(df)


   유저 아이디  방문한 레스토랑 아이디                 방문일시  userRating  travelTime  rating  \
0    2001            19  2024-10-27T15:45:00         5.0           1     4.5   

  menu1 menu2 menu3  category_id1 category_id2  
0    커피  None  None             7         None  


In [6]:
df

,유저 아이디,방문한 레스토랑 아이디,방문일시,userRating,travelTime,rating,menu1,menu2,menu3,category_id1,category_id2
0,2001,19,2024-10-27T15:45:00,5.0,1,4.5,커피,None,None,7,None


In [7]:
import pandas as pd
import json
from datetime import datetime, timedelta  # datetime 모듈 임포트
with open('./reviewlog.json', 'r', encoding='utf-8') as file:
    review_data = json.load(file)


# 방문일시를 datetime 형식으로 변환
df['방문일시'] = pd.to_datetime(df['방문일시'])

df['방문여부'] = 0
df['reviewrating'] = None

# reviewlog 조건을 충족하는지 확인하는 함수 정의
def check_visit_and_rating(row):
    for review in review_data:
        author_id = review['author']['id']
        restaurant_id = review['restaurantId']
        created_at = datetime.strptime(review['createdAt'], "%Y-%m-%dT%H:%M:%S")
        
        # 조건에 맞는지 확인
        if (
            row['유저 아이디'] == author_id and
            row['방문한 레스토랑 아이디'] == restaurant_id and
            abs((row['방문일시'] - created_at).days) <= 1
        ):
            # 조건을 만족할 때 방문여부를 1로 하고, reviewrating 컬럼에 rating 값을 저장
            row['reviewrating'] = review.get('rating')
            row['방문여부'] = 1
            return row

    return row

# 모든 행에 대해 check_visit_and_rating 함수를 적용하여 방문여부와 reviewrating 컬럼을 생성
df = df.apply(check_visit_and_rating, axis=1)


# 결과 출력
df

,유저 아이디,방문한 레스토랑 아이디,방문일시,userRating,travelTime,rating,menu1,menu2,menu3,category_id1,category_id2,방문여부,reviewrating
0,2001,19,2024-10-27 15:45:00,5.0,1,4.5,커피,None,None,7,None,1,4.5


In [8]:
processed_df = df.groupby(['유저 아이디', '방문한 레스토랑 아이디']).apply(lambda group: pd.Series({
    '방문일시': group['방문일시'].max(),
    'userRating': group['userRating'].mean(),
    'travelTime': group['travelTime'].iloc[0],
    'rating': group['rating'].iloc[0],
    'menu1': group['menu1'].iloc[0],
    'menu2': group['menu2'].iloc[0],
    'menu3': group['menu3'].iloc[0],
    'category_id1': group['category_id1'].iloc[0],
    'category_id2': group['category_id2'].iloc[0],
    '평균 reviewRating': group['reviewrating'].mean(),
    '가장 최근 reviewRating': group.sort_values('방문일시', ascending=False)['reviewrating'].iloc[0],
    '방문여부': group['방문여부'].iloc[0]
})).reset_index()

# 결과 출력
processed_df

C:\Users\LG\AppData\Local\Temp\ipykernel_31420\1661343269.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  processed_df = df.groupby(['유저 아이디', '방문한 레스토랑 아이디']).apply(lambda group: pd.Series({


,유저 아이디,방문한 레스토랑 아이디,방문일시,userRating,travelTime,rating,menu1,menu2,menu3,category_id1,category_id2,평균 reviewRating,가장 최근 reviewRating,방문여부
0,2001,19,2024-10-27 15:45:00,5.0,1,4.5,커피,None,None,7,None,4.5,4.5,1


In [9]:
processed_df['만족여부']=None#자연어처리할거임
processed_df

,유저 아이디,방문한 레스토랑 아이디,방문일시,userRating,travelTime,rating,menu1,menu2,menu3,category_id1,category_id2,평균 reviewRating,가장 최근 reviewRating,방문여부,만족여부
0,2001,19,2024-10-27 15:45:00,5.0,1,4.5,커피,None,None,7,None,4.5,4.5,1,None


In [ ]:
가장 최근의 리뷰 점수
가장 최악의 리뷰 점수
가장 최악의 리뷰일 때 리뷰평점
가장 최상의 리뷰 점수
가장 최상의 리뷰일 때 리뷰점수